In [ ]:
# #variavel global
language='pt'
# #Instalando whisper
!pip install git+https://github.com/openai/whisper.git -q
# #Instalando biblioteca da openai
!pip install openai==0.28
# #Biblioteca para sintetizar a voz
!pip install gTTS



from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode
from gtts import gTTS
import openai
import whisper


gtts_object = gTTS(text='Bem vindo ao seu tira  dúvidas, qual a sua dúvida de hoje ?', lang=language, slow=False)
gtts_welcome_path = "/content/welcome.wav"
gtts_object.save(gtts_welcome_path)
display(Audio(gtts_welcome_path, autoplay=True))  

#Funcao em js on premisse onde habilita para escutar audios no navegador
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=10):
    display(Javascript(RECORD))
     #executar o codigo em js e formatando o paramentro s
    js_result = output.eval_js('record(%s)' % (sec * 1000))
    audio = b64decode(js_result.split(',')[1])
    file_name = 'request_audio.wav'
    with open(file_name, 'wb') as f:
        f.write(audio)
    return f'/content/{file_name}'

print('Ouvindo..... \n')
record_file = record()
# display(Audio(record_file, autoplay=True))    
display(Audio(record_file, autoplay=False))  


gtts_object = gTTS(text='Certo, aguarde um momento', lang=language, slow=False)
gtts_await_time_path = "/content/gtts_await_time.wav"
gtts_object.save(gtts_await_time_path)
display(Audio(gtts_await_time_path, autoplay=True))  



#Pode ser o tiny small largel ou outro modelo 
model = whisper.load_model("small")

#atribuindo ao result o audio gravado anteriormente e transcrevendo ele 
#FP16 = false por causa do tipo do audio que estamos utilizando e definimos qual o idioma 
result = model.transcribe(record_file, fp16=False, language=language)

#criando uma nova variavel para receber o texto de result
transcription = result["text"]

print('Escrevendo..... \n')
print(transcription)



#definindo nossa chave de acesso
openai.api_key = ''

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[ { "role": "user","content": transcription } ]
)

chatgpt_response = response.choices[0].message.content
print(chatgpt_response)



gtts_object = gTTS(text='Pronto     ', lang=language, slow=False)
gtts_await_time = "/content/gtts_await_time.wav"
gtts_object.save(gtts_await_time)
display(Audio(gtts_await_time, autoplay=True))  


gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)
display(Audio(response_audio, autoplay=True))